### Data Collection





In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
import os
import json
from requests_oauthlib import OAuth1Session
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#from newsapi import NewsApiClient
from pathlib import Path
import tweepy as tw
%matplotlib inline
load_dotenv()

[nltk_data] Downloading package vader_lexicon to C:\Users\Straw
[nltk_data]     Hat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
consumer_key=os.environ.get("TWITTER_API_KEY")
consumer_secret=os.environ.get("TWITTER_SECERET_API_KEY")
access_token=os.environ.get("ACCESS_TOKEN")
access_token_secret=os.environ.get("ACCESS_TOKEN_SECRET")
bearer_token =os.environ.get("TWITTER_BEARER_TOKEN")

In [4]:
import requests

client = tw.Client( bearer_token, 
                        consumer_key, 
                        consumer_secret, 
                        access_token, 
                        access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

In [5]:
# Define query
query = 'from:zerohedge -is:retweet'

# get max. 10 tweets
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [6]:
# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data) 

In [7]:
#twitter API only limited to 7 days of feeds
df

,created_at,id,text,author_id
0,2022-03-12T17:00:16.000Z,1502690952988737546,2.5 Million Now Seeking Refuge From Russia's I...,18856867
1,2022-03-12T16:57:32.000Z,1502690263306846215,Cohen’s Point72 to Redeem $750 Million From He...,18856867
2,2022-03-12T16:55:18.000Z,1502689701861502976,*RUSSIA CENTRAL BANK EXTENDS STOCK TRADING HAL...,18856867
3,2022-03-12T16:30:19.000Z,1502683414943735814,Global Diesel Shortage Raises Risk Of Even Gre...,18856867
4,2022-03-12T16:01:14.000Z,1502676096227201032,UN Condemns Meta For Allowing 'Hate Speech' Ag...,18856867
...,...,...,...,...
95,2022-03-11T16:30:37.000Z,1502321103884148736,Four VLCCs have been booked to load Russian Ur...,18856867
96,2022-03-11T16:30:12.000Z,1502320999378923524,Bitcoin Slides On Report Russians Seek To Liqu...,18856867
97,2022-03-11T16:25:14.000Z,1502319750470328323,Only 18% Of Americans Say Their Wages Are Keep...,18856867
98,2022-03-11T16:15:25.000Z,1502317279673032707,"RUSSIA AIRSTRIKES IN WESTERN UKRAINE, INCLUDIN...",18856867


In [8]:
# Instantiate the lemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))

In [9]:
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

In [10]:
tokenizer(df.iloc[0]['text'])

['million',
 'now',
 'seeking',
 'refuge',
 'from',
 'russia',
 'invasion',
 'httpstcolwadlokq']

In [11]:
df['tokens'] = df['text'].apply(tokenizer)
df

,created_at,id,text,author_id,tokens
0,2022-03-12T17:00:16.000Z,1502690952988737546,2.5 Million Now Seeking Refuge From Russia's I...,18856867,"[million, now, seeking, refuge, from, russia, ..."
1,2022-03-12T16:57:32.000Z,1502690263306846215,Cohen’s Point72 to Redeem $750 Million From He...,18856867,"[cohens, point, to, redeem, million, from, hed..."
2,2022-03-12T16:55:18.000Z,1502689701861502976,*RUSSIA CENTRAL BANK EXTENDS STOCK TRADING HAL...,18856867,"[russia, central, bank, extends, stock, tradin..."
3,2022-03-12T16:30:19.000Z,1502683414943735814,Global Diesel Shortage Raises Risk Of Even Gre...,18856867,"[global, diesel, shortage, raise, risk, of, ev..."
4,2022-03-12T16:01:14.000Z,1502676096227201032,UN Condemns Meta For Allowing 'Hate Speech' Ag...,18856867,"[un, condemns, meta, for, allowing, hate, spee..."
...,...,...,...,...,...
95,2022-03-11T16:30:37.000Z,1502321103884148736,Four VLCCs have been booked to load Russian Ur...,18856867,"[four, vlccs, have, been, booked, to, load, ru..."
96,2022-03-11T16:30:12.000Z,1502320999378923524,Bitcoin Slides On Report Russians Seek To Liqu...,18856867,"[bitcoin, slide, on, report, russian, seek, to..."
97,2022-03-11T16:25:14.000Z,1502319750470328323,Only 18% Of Americans Say Their Wages Are Keep...,18856867,"[only, of, american, say, their, wage, are, ke..."
98,2022-03-11T16:15:25.000Z,1502317279673032707,"RUSSIA AIRSTRIKES IN WESTERN UKRAINE, INCLUDIN...",18856867,"[russia, airstrikes, in, western, ukraine, inc..."


In [12]:
import spacy
from spacy import displacy

In [13]:
nlp = spacy.load('en_core_web_sm')

In [14]:
text = ' '.join(df['text'])
#text = text.replace("$" , "")
text

'2.5 Million Now Seeking Refuge From Russia\'s Invasion https://t.co/Lw8Ad3lokq Cohen’s Point72 to Redeem 750 Million From Hedge Fund Melvin *RUSSIA CENTRAL BANK EXTENDS STOCK TRADING HALT UNTIL MARCH 18 Global Diesel Shortage Raises Risk Of Even Greater Oil Price Spike https://t.co/rcuFXF2DvO UN Condemns Meta For Allowing \'Hate Speech\' Against Russia\xa0 https://t.co/10wisvvjgO Russia Threatens Attack On NATO Weapons Shipments To Ukraine: "Legitimate Targets" https://t.co/Mx3a18o9Dx FRENCH PRESIDENCY OFFICIAL SAYS, IN CALL WITH MACRON, SCHOLZ, THAT RUSSIA\'S PUTIN DID NOT APPEAR READY TO END WAR A Visual Guide To Europe\'s Member States https://t.co/G1uCH3lAO1 Rickards: The Sanctions Boomerang &amp; Putin\'s Options https://t.co/o7DdbGid05 RUSSIA\'S RYABKOV SAYS RUSSIAN FEDERATION IS SATISFIED WITH THE DIALOGUE WITH CHINA, IT IS A RELIABLE PARTNER OF RUSSIA UKRAINE\'S ZELENSKIY SAYS WEST NOT INVOLVED ENOUGH IN PEACE NEGOTIATIONS\n\nUKRAINE\'S ZELENSKIY SAYS HE DOESN\'T SEE COMMON CO

In [15]:
doc = nlp(text)

In [16]:
displacy.render(doc, style='ent')

In [17]:
from collections import Counter

# List all Entities
org_list = []

for ent in doc.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)


org_list = Counter(org_list).most_common(15)
df_org = pd.DataFrame(org_list, columns = ['text', 'count'])

df_org

,text,count
0,UN,2
1,JPM,2
2,U.N.,2
3,Invasion https://t.co/Lw8Ad3lokq Cohen’s Point72,1
4,The Sanctions Boomerang &,1
5,Options https://t.co/o7DdbGid05 RUSSIA'S,1
6,NASA,1
7,Crucial To Life On Mars https://t.co/66Gz3L0u4...,1
8,Updates Search Engine,1
9,Collapse,1


In [18]:
df.to_csv('tweets1.csv')

In [19]:
#second tweet collection 
# Define query
query = 'from:InvestorsLive -is:retweet'

# get max. 10 tweets
tweets2 = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [20]:
# Save data as dictionary
tweets_dict2 = tweets2.json() 

# Extract "data" value from dictionary
tweets_data2 = tweets_dict2['data'] 

# Transform to pandas Dataframe
df2 = pd.json_normalize(tweets_data2) 

df2

,id,created_at,text,author_id
0,1502675863049052163,2022-03-12T16:00:18.000Z,Pay attention to key levels 📊 https://t.co/Sod...,15971805
1,1502657337160515584,2022-03-12T14:46:41.000Z,@OddStockTrader @IU_Chat Well if he is going t...,15971805
2,1502656112897712129,2022-03-12T14:41:49.000Z,@OddStockTrader @IU_Chat You visiting again? H...,15971805
3,1502480636879253508,2022-03-12T03:04:33.000Z,"@RedRockTrader1 Yes, I missed the short after ...",15971805
4,1502480073798078468,2022-03-12T03:02:18.000Z,"@Rebelo49 Doesn't matter how many fills, per s...",15971805
...,...,...,...,...
95,1501982175318749185,2022-03-10T18:03:50.000Z,$ISPO I liked it last time when it went straig...,15971805
96,1501975369422888972,2022-03-10T17:36:48.000Z,She's a beaut Clark\n\n$AGRI https://t.co/8jfT...,15971805
97,1501966106084397057,2022-03-10T16:59:59.000Z,1 by 1 they go,15971805
98,1501965733965803523,2022-03-10T16:58:30.000Z,$ENSV https://t.co/FxDmd5dytF,15971805


In [21]:
def tokenizer(text2):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text2)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

In [22]:
tokenizer(df2.iloc[0]['text'])

['pay', 'attention', 'to', 'key', 'level', 'httpstcosodqvcqae']

In [23]:
df2['tokens'] = df2['text'].apply(tokenizer)
df2

,id,created_at,text,author_id,tokens
0,1502675863049052163,2022-03-12T16:00:18.000Z,Pay attention to key levels 📊 https://t.co/Sod...,15971805,"[pay, attention, to, key, level, httpstcosodqv..."
1,1502657337160515584,2022-03-12T14:46:41.000Z,@OddStockTrader @IU_Chat Well if he is going t...,15971805,"[oddstocktrader, iuchat, well, if, he, is, goi..."
2,1502656112897712129,2022-03-12T14:41:49.000Z,@OddStockTrader @IU_Chat You visiting again? H...,15971805,"[oddstocktrader, iuchat, you, visiting, again,..."
3,1502480636879253508,2022-03-12T03:04:33.000Z,"@RedRockTrader1 Yes, I missed the short after ...",15971805,"[redrocktrader, yes, i, missed, the, short, af..."
4,1502480073798078468,2022-03-12T03:02:18.000Z,"@Rebelo49 Doesn't matter how many fills, per s...",15971805,"[rebelo, doesnt, matter, how, many, fill, per,..."
...,...,...,...,...,...
95,1501982175318749185,2022-03-10T18:03:50.000Z,$ISPO I liked it last time when it went straig...,15971805,"[ispo, i, liked, it, last, time, when, it, wen..."
96,1501975369422888972,2022-03-10T17:36:48.000Z,She's a beaut Clark\n\n$AGRI https://t.co/8jfT...,15971805,"[shes, a, beaut, clarkagri, httpstcojftjxecwc]"
97,1501966106084397057,2022-03-10T16:59:59.000Z,1 by 1 they go,15971805,"[by, they, go]"
98,1501965733965803523,2022-03-10T16:58:30.000Z,$ENSV https://t.co/FxDmd5dytF,15971805,"[ensv, httpstcofxdmddytf]"


In [24]:
nlp = spacy.load('en_core_web_sm')

In [25]:
text2 = ' '.join(df2['text'])
#text2 = text2.replace("$" , "")
text2

'Pay attention to key levels 📊 https://t.co/SodQ8VCQae @OddStockTrader @IU_Chat Well if he is going to meet you at 11am that means ~1pm just FYI \n\ncc: @swedepilot @OddStockTrader @IU_Chat You visiting again? He\'s there. @RedRockTrader1 Yes, I missed the short after the liquidity trap played out so well, pissed! Kept meaning to fade it past 2 days given the 10 exhaustion and post squeeze flush we went over in the room -  but was focused on the oil and gas fade instead. @Rebelo49 Doesn\'t matter how many fills, per share commission same regardless. Sometimes you just have to ask for the VWAP squeeze out to replace. Nice dunk AHs - big cover came in at close less timid shorts for Monday likely gap down HYMC https://t.co/PQuRKm1Efe @YSRSFS @kycefn Request to upgrade via your broker - they can add on via DAS. @YSRSFS You have to pay for it. Most have it. @fel0ncod3 Correct. The 750k bidder is just there while they sell hidden (showing 100 shares) at .65 - fwiw\n\nBe careful falling for t

In [26]:
doc2 = nlp(text2)

In [27]:
displacy.render(doc2, style='ent')

In [28]:
df2.to_csv('tweets2.csv')

In [29]:
from collections import Counter

# List all Entities
org_list2 = []

for ent in doc2.ents:
    if ent.label_ == 'ORG':
        org_list2.append(ent.text)


org_list2 = Counter(org_list2).most_common(15)
df_org2 = pd.DataFrame(org_list2, columns = ['text', 'count'])

df_org2

,text,count
0,HYMC,5
1,AGRI,4
2,WULF,2
3,DAS,1
4,@fel0ncod3,1
5,750k,1
6,GDS,1
7,Learning &,1
8,@CUMBACKTIME,1
9,iPhone,1


In [30]:
#Third tweet collection 
# Define query
query = 'from:markflowchatter -is:retweet'

# get max. 10 tweets
tweets3 = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [31]:
# Save data as dictionary
tweets_dict3 = tweets3.json() 

# Extract "data" value from dictionary
tweets_data3 = tweets_dict3['data'] 

# Transform to pandas Dataframe
df3 = pd.json_normalize(tweets_data3) 

In [32]:
def tokenizer(text3):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text3)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

In [33]:
tokenizer(df3.iloc[0]['text'])

['bmrt', 'amptrades', 'elkwood', 'work', 'him', 'too', 'hard']

In [34]:
df3['tokens'] = df3['text'].apply(tokenizer)
df3

,created_at,id,text,author_id,tokens
0,2022-03-11T00:15:22.000Z,1502075671970992133,@b_m0rt @Amp_Trades @elkwood66 works him too hard,62562916,"[bmrt, amptrades, elkwood, work, him, too, hard]"
1,2022-03-10T15:36:48.000Z,1501945171625213952,https://t.co/tMpqKmqf0K,62562916,[httpstcotmpqkmqfk]
2,2022-03-09T14:11:50.000Z,1501561401361543179,"CEO of $DKNG tweet, maybe the FT $DIS has legs...",62562916,"[ceo, of, dkng, tweet, maybe, the, ft, dis, ha..."
3,2022-03-08T13:31:34.000Z,1501188878404890629,$DKNG\nDisney plots sports betting push in bid...,62562916,"[dkngdisney, plot, sport, betting, push, in, b..."
4,2022-03-07T01:50:40.000Z,1500650105103433728,This puts it all in perspective.\nAlthough you...,62562916,"[this, put, it, all, in, perspectivealthough, ..."
5,2022-03-07T00:08:53.000Z,1500624491319332866,"Ryan Cohen, chairman on $GME takes a 9.9% stak...",62562916,"[ryan, cohen, chairman, on, gme, take, a, stak..."


In [35]:
text3 = ' '.join(df3['text'])
#text3 = text3.replace("$" , "")
text3

"@b_m0rt @Amp_Trades @elkwood66 works him too hard https://t.co/tMpqKmqf0K CEO of DKNG tweet, maybe the FT DIS has legs https://t.co/IpHgmxMHKB DKNG\nDisney plots sports betting push in bid to revitalise ESPN -\nCable sports network considers striking licensing deals with bookmakers  \n\nDisney has a 6 percent stake in DraftKing DKNG which would be the obvious target\n\n https://t.co/IVGvkd4urb This puts it all in perspective.\nAlthough you left out the cryin' Bill theory https://t.co/qRj20vJBis Ryan Cohen, chairman on GME takes a 9.9% stake in BBBY\n\nhttps://t.co/4V7ObXqTmy"

In [36]:
doc3 = nlp(text3)

In [37]:
displacy.render(doc3, style='ent')

In [38]:
df3.to_csv('tweets3.csv')

In [39]:
from collections import Counter

# List all Entities
org_list3 = []

for ent in doc3.ents:
    if ent.label_ == 'ORG':
        org_list3.append(ent.text)


org_list3 = Counter(org_list3).most_common(15)
df_org3 = pd.DataFrame(org_list3, columns = ['text', 'count'])

df_org3

,text,count
0,@b_m0rt @Amp_Trades @elkwood66,1
1,DKNG,1
2,DIS,1
3,ESPN -\nCable,1
4,Disney,1
5,GME,1
6,BBBY,1


In [40]:
#fourth tweet collection 
# Define query
query = 'from:OptionsHawk -is:retweet'

# get max. 10 tweets
tweets4 = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [41]:
# Save data as dictionary
tweets_dict4 = tweets4.json() 

# Extract "data" value from dictionary
tweets_data4 = tweets_dict4['data'] 

# Transform to pandas Dataframe
df4 = pd.json_normalize(tweets_data4) 

df4

,text,id,created_at,author_id
0,Wordle 265 6/6\n\n⬛⬛⬛⬛⬛\n⬛⬛⬛🟨⬛\n🟨⬛🟨⬛⬛\n⬛🟩🟩⬛⬛\n...,1502465745845985285,2022-03-12T02:05:22.000Z,61342056
1,@gldfngr38 260 and took about 5+ hours for 3-4...,1502427002447482885,2022-03-11T23:31:25.000Z,61342056
2,@johnfrenchbmore Soaked it first to remove the...,1502426908092514312,2022-03-11T23:31:03.000Z,61342056
3,Never smoked a corn beef brisket before but da...,1502424048416038915,2022-03-11T23:19:41.000Z,61342056
4,Always try and find a beer based on how the ma...,1502404635335737358,2022-03-11T22:02:33.000Z,61342056
...,...,...,...,...
95,Options volume has come down in Indiv. names b...,1501593766808932356,2022-03-09T16:20:26.000Z,61342056
96,"RY, BNS, BMO - April puts in Canadian Banks lo...",1501586860593790980,2022-03-09T15:53:00.000Z,61342056
97,"@NielSagaert @FutureFetish007 The worms, the s...",1501585228648833024,2022-03-09T15:46:31.000Z,61342056
98,"@MichaelMulin Your numbers are wrong, no1 care...",1501582690281271303,2022-03-09T15:36:26.000Z,61342056


In [42]:
def tokenizer(text4):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text4)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

In [43]:
tokenizer(df4.iloc[0]['text'])

['wordle', 'streak', 'life']

In [44]:
df4['tokens'] = df4['text'].apply(tokenizer)
df4

,text,id,created_at,author_id,tokens
0,Wordle 265 6/6\n\n⬛⬛⬛⬛⬛\n⬛⬛⬛🟨⬛\n🟨⬛🟨⬛⬛\n⬛🟩🟩⬛⬛\n...,1502465745845985285,2022-03-12T02:05:22.000Z,61342056,"[wordle, streak, life]"
1,@gldfngr38 260 and took about 5+ hours for 3-4...,1502427002447482885,2022-03-11T23:31:25.000Z,61342056,"[gldfngr, and, took, about, hour, for, pounder]"
2,@johnfrenchbmore Soaked it first to remove the...,1502426908092514312,2022-03-11T23:31:03.000Z,61342056,"[johnfrenchbmore, soaked, it, first, to, remov..."
3,Never smoked a corn beef brisket before but da...,1502424048416038915,2022-03-11T23:19:41.000Z,61342056,"[never, smoked, a, corn, beef, brisket, before..."
4,Always try and find a beer based on how the ma...,1502404635335737358,2022-03-11T22:02:33.000Z,61342056,"[always, try, and, find, a, beer, based, on, h..."
...,...,...,...,...,...
95,Options volume has come down in Indiv. names b...,1501593766808932356,2022-03-09T16:20:26.000Z,61342056,"[option, volume, ha, come, down, in, indiv, na..."
96,"RY, BNS, BMO - April puts in Canadian Banks lo...",1501586860593790980,2022-03-09T15:53:00.000Z,61342056,"[ry, bns, bmo, april, put, in, canadian, bank,..."
97,"@NielSagaert @FutureFetish007 The worms, the s...",1501585228648833024,2022-03-09T15:46:31.000Z,61342056,"[nielsagaert, futurefetish, the, worm, the, so..."
98,"@MichaelMulin Your numbers are wrong, no1 care...",1501582690281271303,2022-03-09T15:36:26.000Z,61342056,"[michaelmulin, your, number, are, wrong, no, c..."


In [45]:
text4 = ' '.join(df4['text'])
text4

'Wordle 265 6/6\n\n⬛⬛⬛⬛⬛\n⬛⬛⬛\U0001f7e8⬛\n\U0001f7e8⬛\U0001f7e8⬛⬛\n⬛\U0001f7e9\U0001f7e9⬛⬛\n⬛\U0001f7e9\U0001f7e9\U0001f7e9\U0001f7e9\n\U0001f7e9\U0001f7e9\U0001f7e9\U0001f7e9\U0001f7e9\n\nStreak lives! @gldfngr38 260 and took about 5+ hours for 3-4 pounder @johnfrenchbmore Soaked it first to remove the salt Never smoked a corn beef brisket before but damn that’s good https://t.co/cGlweT63wk Always try and find a beer based on how the market ends the week..this will do https://t.co/g9bzlyItue It’s going to be a barn burner of a close! https://t.co/LSh9FrMnOb Are we starting to see a shift to fade morning and buy afternoons (institutions) ?\n\nSeems different this week and VIX continues to yawn into morning dips Smoking a corn beef brisket on the agenda the remainder of the day.... Euro Stoxx FEZ vote of confidence sells 1500 Jan. 2024 38 puts for 5 @fundmyfund But then you run into raising unsocialized weirdos Probably need @Kalshi to put up a Yes/No contract for "Will Chinese-domicile

In [46]:
doc4 = nlp(text4)

In [47]:
displacy.render(doc4, style='ent')

In [48]:
df4.to_csv('tweets4.csv')

In [49]:
from collections import Counter

# List all Entities
org_list4 = []

for ent in doc4.ents:
    if ent.label_ == 'ORG':
        org_list4.append(ent.text)


org_list4 = Counter(org_list4).most_common(15)
df_org4 = pd.DataFrame(org_list4, columns = ['text', 'count'])

df_org4

,text,count
0,⬛,8
1,MLB,2
2,AMR,2
3,IWM,2
4,BMY,1
5,Tech,1
6,HON,1
7,INCY,1
8,Hydrogen,1
9,Solar,1


In [57]:
print(df[df['text'].str.contains('$', regex=False)])


                  created_at                   id  \
1   2022-03-12T16:57:32.000Z  1502690263306846215   
70  2022-03-11T18:17:03.000Z  1502347887795224590   
71  2022-03-11T18:15:27.000Z  1502347486773624837   

                                                 text author_id  \
1   Cohen’s Point72 to Redeem $750 Million From He...  18856867   
70  *83 COUNTERPARTIES TAKE $1.558 TLN AT FED REVE...  18856867   
71  *SEC AWARDS $14M TO WHISTLEBLOWER WHO EXPOSED ...  18856867   

                                               tokens  
1   [cohens, point, to, redeem, million, from, hed...  
70  [counterparties, take, tln, at, fed, reverse, ...  
71  [sec, award, m, to, whistleblower, who, expose...  


In [60]:
print(df[df['id'].str.contains('1502690263306846215'and '1502347887795224590' and '1502347486773624837', regex=False)])

                  created_at                   id  \
71  2022-03-11T18:15:27.000Z  1502347486773624837   

                                                 text author_id  \
71  *SEC AWARDS $14M TO WHISTLEBLOWER WHO EXPOSED ...  18856867   

                                               tokens  
71  [sec, award, m, to, whistleblower, who, expose...  
